In [1]:
# Opracowane na podstawie https://github.com/FluxML/model-zoo/blob/3e91af32ebfad628b616618b11bfff2f9f519bec/vision/conv_mnist/conv_mnist.jl
using MLDatasets, Flux
train_data = MLDatasets.MNIST(split=:train)
test_data  = MLDatasets.MNIST(split=:test)

function loader(data; batchsize::Int=1)
    x4dim = reshape(data.features, 28, 28, 1, :) # insert trivial channel dim
    yhot  = Flux.onehotbatch(data.targets, 0:9)  # make a 10×60000 OneHotMatrix
    Flux.DataLoader((x4dim, yhot); batchsize, shuffle=true)
end

net = Chain(
    Conv((3, 3), 1 => 6,  relu),
    MaxPool((2, 2)),
    Conv((3, 3), 6 => 16, relu),
    MaxPool((2, 2)),
    Flux.flatten,
    Dense(400 => 84, relu), 
    Dense(84 => 10, identity),
)

Chain(
  Conv((3, 3), 1 => 6, relu),           # 60 parameters
  MaxPool((2, 2)),
  Conv((3, 3), 6 => 16, relu),          # 880 parameters
  MaxPool((2, 2)),
  Flux.flatten,
  Dense(400 => 84, relu),               # 33_684 parameters
  Dense(84 => 10),                      # 850 parameters
)                   # Total: 8 arrays, 35_474 parameters, 139.773 KiB.

In [2]:
x1, y1 = first(loader(train_data)); # (28×28×1×1 Array{Float32, 3}, 10×1 OneHotMatrix(::Vector{UInt32}))
y1hat = net(x1)
@show hcat(Flux.onecold(y1hat, 0:9), Flux.onecold(y1, 0:9))

using Statistics: mean  # standard library
function loss_and_accuracy(model, data)
    (x,y) = only(loader(data; batchsize=length(data)))
    ŷ = model(x)
    loss = Flux.logitcrossentropy(ŷ, y)  # did not include softmax in the model
    acc = round(100 * mean(Flux.onecold(ŷ) .== Flux.onecold(y)); digits=2)
    (; loss, acc, split=data.split)  # return a NamedTuple
end

@show loss_and_accuracy(net, test_data);  # accuracy about 10%, before training

train_log = []
settings = (;
    eta = 1e-2,
    epochs = 3,
    batchsize = 100,
)

opt_state = Flux.setup(Descent(settings.eta), net);

hcat(Flux.onecold(y1hat, 0:9), Flux.onecold(y1, 0:9)) = [0 8]
loss_and_accuracy(net, test_data) = (loss = 2.3211448f0, acc = 11.16, split = :test)


In [3]:

grads = []
for epoch in 1:settings.epochs
    @time for (x,y) in loader(train_data, batchsize=settings.batchsize)
        loader(train_data, batchsize=settings.batchsize)
        grads = Flux.gradient(model -> Flux.logitcrossentropy(model(x), y), net)
        Flux.update!(opt_state, net, grads[1])
    end
    loss, acc, _ = loss_and_accuracy(net, train_data)
    test_loss, test_acc, _ = loss_and_accuracy(net, test_data)
    @info epoch acc test_acc
    nt = (; epoch, loss, acc, test_loss, test_acc) 
    push!(train_log, nt)
end

 37.983814 seconds (24.46 M allocations: 9.727 GiB, 3.66% gc time, 66.51% compilation time)


┌ Info: 1
│   acc = 88.18
└   test_acc = 88.9


 11.462788 seconds (462.49 k allocations: 8.200 GiB, 9.69% gc time)


┌ Info: 2
│   acc = 91.74
└   test_acc = 92.26


 10.595408 seconds (462.48 k allocations: 8.200 GiB, 6.42% gc time)


┌ Info: 3
│   acc = 93.39
└   test_acc = 93.87


In [4]:
x1, y1 = first(loader(train_data)); # (28×28×1×1 Array{Float32, 3}, 10×1 OneHotMatrix(::Vector{UInt32}))
y1hat = net(x1)
@show hcat(Flux.onecold(y1hat, 0:9), Flux.onecold(y1, 0:9))
@show loss_and_accuracy(net, train_data);

hcat(Flux.onecold(y1hat, 0:9), Flux.onecold(y1, 0:9)) = [1 1]
(84, 400)
Array{Float32, 4}
loss_and_accuracy(net, train_data) = (loss = 0.2182844f0, acc = 93.39, split = :train)
